In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Step 1: Define the Encoder class
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size)

    def forward(self, input_seq):
        output, (hidden, cell) = self.lstm(input_seq)
        print(output.shape, hidden.shape, cell.shape)
        print('output', output[-1])
        print('hidden', hidden[-1]
        print(pato)
        return hidden, cell

# Step 2: Define the Decoder class
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(output_size, hidden_size)
        self.fc_out = nn.Linear(hidden_size, output_size)

    def forward(self, input_seq, hidden, cell):
        output, (hidden, cell) = self.lstm(input_seq, (hidden, cell))
        prediction = self.fc_out(output)
        return prediction, hidden, cell

# Step 3: Define the Seq2Seq model class with Teacher Forcing
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, source_seq, target_seq_len, teacher_forcing_ratio=0.5):
        batch_size = source_seq.shape[1]
        output_size = self.decoder.fc_out.out_features
        outputs = torch.zeros(target_seq_len, batch_size, output_size).to(source_seq.device)

        hidden, cell = self.encoder(source_seq)

        decoder_input = torch.zeros(1, batch_size, output_size).to(source_seq.device)

        for t in range(target_seq_len):
            output, hidden, cell = self.decoder(decoder_input, hidden, cell)
            outputs[t] = output
            use_teacher_forcing = np.random.random() < teacher_forcing_ratio
            decoder_input = output if not use_teacher_forcing else source_seq[t].unsqueeze(0)

        return outputs

# Step 4: Generate Data with multiple features
def generate_data(seq_len=50, n_samples=1000, n_features=1):
    X = np.linspace(0, 100, n_samples)
    y = np.sin(X)  # Change this to any other function for experimentation
    
    data = []
    for i in range(len(y) - seq_len):
        feature_data = [np.sin(X[i:i+seq_len] + np.random.randn()*0.1) for _ in range(n_features)]
        data.append(np.stack(feature_data, axis=-1))

    data = np.array(data)
    
    train_size = int(0.8 * data.shape[0])
    X_train = data[:train_size]
    X_test = data[train_size:]

    return torch.tensor(X_train, dtype=torch.float32).to(device), torch.tensor(X_test, dtype=torch.float32).to(device)

# Step 5: Training the Seq2Seq model
def train_seq2seq(model, X_train, epochs=100, learning_rate=0.001, teacher_forcing_ratio=0.5):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(epochs):
        optimizer.zero_grad()

        outputs = model(X_train.permute(1, 0, 2), X_train.shape[1], teacher_forcing_ratio)
        
        loss = criterion(outputs, X_train.permute(1, 0, 2))
        
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Step 6: Evaluation with plotting
def evaluate_seq2seq(model, X_test):
    model.eval()
    with torch.no_grad():
        predictions = model(X_test.permute(1, 0, 2), X_test.shape[1], teacher_forcing_ratio=0.0)
        predictions = predictions.permute(1, 0, 2).cpu().numpy()  # Convert to numpy for plotting
        X_test = X_test.cpu().numpy()  # Convert to numpy for plotting

        # Plot the actual and predicted sequences
        plt.figure(figsize=(10, 6))
        plt.plot(X_test[0, :, 0], label="Actual Sequence (Feature 1)", color='green')
        plt.plot(predictions[0, :, 0], label="Predicted Sequence (Feature 1)", linestyle=":", color='red')
        plt.title("Actual vs Predicted Sequence")
        plt.xlabel("Time step")
        plt.ylabel("Value")
        plt.legend()
        plt.show()

        # plot the actual and predicted sequences
        plt.figure(figsize=(10, 6))
        plt.plot(X_test[0, :, 1], label="Actual Sequence (Feature 2)", color='green')
        plt.plot(predictions[0, :, 1], label="Predicted Sequence (Feature 2)", linestyle=":", color='red')
        plt.title("Actual vs Predicted Sequence")
        plt.xlabel("Time step")
        plt.ylabel("Value")
        plt.legend()
        plt.show()

# Step 7: Putting it all together
if __name__ == "__main__":
    # Generate data with multiple features
    n_features = 10  # Number of features
    seq_len = 50
    X_train, X_test = generate_data(seq_len=seq_len, n_samples=1100, n_features=n_features)

    # Define the model parameters
    input_size = n_features
    hidden_size = 64
    output_size = n_features

    # Initialize Encoder, Decoder, and Seq2Seq model
    encoder = Encoder(input_size, hidden_size).to(device)
    decoder = Decoder(hidden_size, output_size).to(device)
    seq2seq_model = Seq2Seq(encoder, decoder).to(device)

    # Train the model
    train_seq2seq(seq2seq_model, X_train, epochs=200, learning_rate=0.001, teacher_forcing_ratio=0.5)

    # Evaluate and plot
    evaluate_seq2seq(seq2seq_model, X_test)


Using device: cuda
torch.Size([50, 840, 64]) torch.Size([1, 840, 64]) torch.Size([1, 840, 64])
output tensor([[[-0.0274, -0.0183, -0.0058,  ..., -0.0182, -0.0375,  0.0215],
         [-0.0303, -0.0084, -0.0122,  ..., -0.0176, -0.0338,  0.0237],
         [-0.0189,  0.0017, -0.0298,  ..., -0.0125, -0.0234,  0.0352],
         ...,
         [-0.0655,  0.0309, -0.0562,  ..., -0.0098, -0.0408,  0.0422],
         [-0.0681,  0.0344, -0.0649,  ..., -0.0051, -0.0402,  0.0428],
         [-0.0752,  0.0401, -0.0596,  ...,  0.0065, -0.0469,  0.0426]],

        [[-0.0464, -0.0226, -0.0143,  ..., -0.0255, -0.0580,  0.0309],
         [-0.0503, -0.0094, -0.0247,  ..., -0.0229, -0.0529,  0.0336],
         [-0.0330,  0.0035, -0.0515,  ..., -0.0161, -0.0367,  0.0515],
         ...,
         [-0.0914,  0.0404, -0.0925,  ..., -0.0049, -0.0617,  0.0607],
         [-0.0938,  0.0433, -0.1048,  ...,  0.0025, -0.0604,  0.0613],
         [-0.1022,  0.0512, -0.0976,  ...,  0.0184, -0.0689,  0.0611]],

        [[-0.0

NameError: name 'pato' is not defined

In [ ]:
()